In [1]:
import pandas as pd
import os

%matplotlib inline

In [2]:
def apply_labels(df,table_type):
    '''
    Replace values with human readable lablels.
    '''
    df_label = labels[labels['table'] == table_type]
    for field in df_label['field'].unique():
        newdf = df_label[df_label['field'] == field]
        local_series = pd.Series(newdf['text'].values, index=newdf['value'])
        df[field] = df[field].map(local_series)
    
    return df

In [3]:
# Daysim data
trip = pd.read_csv(r'../../../outputs/daysim/_trip.tsv', sep='\t')
person = pd.read_csv(r'../../../outputs/daysim/_person.tsv', sep='\t')
hh = pd.read_csv(r'../../../outputs/daysim/_household.tsv', sep='\t')

# Add labels
labels = pd.read_csv(r'../../../scripts/summarize/inputs/calibration/variable_labels.csv')

trip = apply_labels(trip,'Trip')
person = apply_labels(person,'Person')
hh = apply_labels(hh,'Household')

In [4]:

special_taz = pd.read_csv(r'../inputs/special_needs_taz.csv')
rgc_taz = pd.read_csv(r'../../../scripts/summarize/inputs/rgc_taz.csv')

In [5]:
# Join RGC, low income, and minority data, based on home location

trip_hh = pd.merge(trip,hh,on='hhno',how='left')

trip_hh = pd.merge(trip_hh, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
trip_hh = pd.merge(trip_hh, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

In [6]:
pd.options.display.float_format = '{:.2f}'.format

## Mode Share
Based on home location

In [10]:
def mode_share(trip_hh):
    """
    Produce trip mode share for low income, minority, and regional populations
    """
    # Regional total
    df_reg = pd.DataFrame(
        trip_hh.groupby('mode').sum()['trexpfac']/trip_hh['trexpfac'].sum()*100)
    df_reg.columns = ['Region']
    df_reg = df_reg.reset_index()
    
    # Low Income
    df = trip_hh[['trexpfac','Low Income','mode']].groupby(['Low Income','mode']).sum().reset_index()
    tot_trips = pd.DataFrame(df.groupby('Low Income').sum()['trexpfac']).reset_index()
    df = pd.merge(df,tot_trips,on='Low Income', suffixes=['_mode','_total'])
    df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])*100

    df_inc = df.pivot(index='Low Income', columns='mode', values='mode_share')
    df_inc.index = ['Other','People of Lower Income']
    
    # Minority
    df = trip_hh[['trexpfac','Minority','mode']].groupby(['Minority','mode']).sum().reset_index()
    tot_trips = pd.DataFrame(df.groupby('Minority').sum()['trexpfac']).reset_index()
    df = pd.merge(df,tot_trips,on='Minority', suffixes=['_mode','_total'])
    df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])*100

    df_race = df.pivot(index='Minority', columns='mode', values='mode_share')
    df_race.index = ['Other','People of Color']
    
    # Merge all dataframes
    df = pd.merge(pd.DataFrame(df_race.loc['People of Color']).reset_index(),
         pd.DataFrame(df_inc.loc['People of Lower Income']).reset_index())
    df = pd.merge(df_reg, df)
    df.index = df['mode']
    df = df.drop('mode', axis=1)
    return df

### All Trips

In [59]:
pd.options.display.float_format = '{:.2f}%'.format
df = mode_share(trip_hh)
df

### Commute Trips

In [15]:
commute_trips = trip_hh[trip_hh['dpurp'] == 'Work']
commute_trips = commute_trips[commute_trips['mode'] != 'School Bus']
df = mode_share(commute_trips)
df

,Region,People of Color,People of Lower Income
mode,,,
Bike,2.32%,2.48%,2.75%
HOV2,9.59%,9.70%,9.58%
HOV3+,4.34%,4.48%,4.39%
SOV,65.40%,63.91%,63.72%
Transit,6.33%,6.70%,6.21%
Walk,12.01%,12.73%,13.36%


### By Regional Growth Center
Households living in the center

#### All Trips

In [21]:
# Transit Mode share by RGC home location
df = trip_hh[['trexpfac','geog_name','mode']].groupby(['geog_name','mode']).sum().reset_index()
tot_rgc = trip_hh[['trexpfac','geog_name']].groupby('geog_name').sum().reset_index()
df = pd.merge(df,tot_rgc,on='geog_name', suffixes=['_mode','_total'])
df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])*100

df = df.pivot(index='geog_name', columns='mode', values='mode_share')
# Trim the number of RGCs
rgc_list = ['Auburn','Bellevue','Bremerton','Everett','Issaquah','Lynnwood','Redmond Downtown',
       'Renton','Seattle Downtown','Seattle Northgate','Seattle University Community',
        'Seattle Uptown', 'Tacoma Downtown']
df.loc[rgc_list]

mode,Bike,HOV2,HOV3+,SOV,School Bus,Transit,Walk
geog_name,,,,,,,
Auburn,2.45%,22.77%,14.12%,32.48%,0.98%,5.63%,21.58%
Bellevue,2.03%,14.57%,7.05%,26.81%,0.86%,6.60%,42.08%
Bremerton,2.85%,17.34%,10.87%,30.20%,2.09%,3.20%,33.46%
Everett,3.25%,15.53%,7.74%,33.04%,0.96%,1.98%,37.49%
Issaquah,0.32%,15.51%,11.71%,53.16%,1.42%,6.96%,10.92%
Lynnwood,2.00%,22.22%,14.13%,33.52%,2.59%,2.17%,23.36%
Redmond Downtown,2.56%,16.67%,8.60%,37.57%,0.93%,6.44%,27.23%
Renton,1.72%,16.86%,9.17%,41.02%,1.31%,6.94%,22.99%
Seattle Downtown,4.14%,9.24%,3.46%,19.88%,0.46%,11.72%,51.09%


#### Commute Trips

In [24]:
# Transit Mode share by RGC home location
df = commute_trips[['trexpfac','geog_name','mode']].groupby(['geog_name','mode']).sum().reset_index()
tot_rgc = commute_trips[['trexpfac','geog_name']].groupby('geog_name').sum().reset_index()
df = pd.merge(df,tot_rgc,on='geog_name', suffixes=['_mode','_total'])
df['mode_share'] = (df['trexpfac_mode']/df['trexpfac_total'])*100

df = df.pivot(index='geog_name', columns='mode', values='mode_share')
# Trim the number of RGCs
rgc_list = ['Auburn','Bellevue','Bremerton','Everett','Issaquah','Lynnwood','Redmond Downtown',
       'Renton','Seattle Downtown','Seattle Northgate','Seattle University Community',
        'Seattle Uptown', 'Tacoma Downtown']
df = df.fillna(0)
df.loc[rgc_list]

mode,Bike,HOV2,HOV3+,SOV,Transit,Walk
geog_name,,,,,,
Auburn,5.15%,9.01%,3.86%,58.80%,7.73%,15.45%
Bellevue,2.56%,5.34%,2.00%,33.60%,9.02%,47.49%
Bremerton,5.23%,8.50%,4.97%,48.50%,3.79%,29.02%
Everett,4.31%,7.33%,2.80%,47.82%,4.20%,33.53%
Issaquah,0.00%,4.17%,1.67%,67.50%,12.50%,14.17%
Lynnwood,2.10%,8.59%,2.80%,59.54%,5.59%,21.38%
Redmond Downtown,3.71%,8.15%,3.08%,50.20%,10.92%,23.94%
Renton,2.62%,8.49%,3.12%,59.28%,10.17%,16.32%
Seattle Downtown,4.44%,4.73%,1.60%,24.64%,11.32%,53.26%


# Trip Ends

Results based on trip end location

### Access to Frequent Transit

In [25]:
df = pd.read_csv(r'../../../outputs/transit/freq_transit_access.csv')
# Join parcel data (transit access) to household recods
df = pd.merge(hh[['hhparcel','hhsize','hhtaz']],df[['PARCELID','dist_frequent']],how='left',left_on='hhparcel',right_on='PARCELID')

# join to geography fiels
df = pd.merge(df, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
df = pd.merge(df, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

In [27]:
# Only include househodls on parcels within 1/2 mile of frequent transit
max_dist = 0.5
df_freq = df[df['dist_frequent'] <= max_dist]

In [ ]:
# Calculate percent of people within that range

In [32]:
df_lowinc = pd.DataFrame(df_freq.groupby('Low Income').sum()['hhsize']/df.groupby('Low Income').sum()['hhsize'])

In [34]:
df_minority = pd.DataFrame(df_freq.groupby('Minority').sum()['hhsize']/df.groupby('Minority').sum()['hhsize'])

In [53]:
reg_share = df_freq.sum()['hhsize']/df.sum()['hhsize']

In [62]:
_df = pd.DataFrame([reg_share,df_minority.loc[1].values[0],df_lowinc.loc[1].values[0]])
_df.index = ['Region','People of Color','People of Low Income']
_df.columns = ['Percent with Access to Frequent Transit Service']
_df['Percent with Access to Frequent Transit Service'] = _df['Percent with Access to Frequent Transit Service']*100
_df

,Percent with Access to Frequent Transit Service
Region,66.95%
People of Color,79.48%
People of Low Income,78.08%


## Percent of People Walking and Biking for Transport

In [6]:
trip_person = pd.merge(trip,person,on=['hhno','pno'], how='left')
person_hh = pd.merge(person,hh[['hhno','hhtaz']],on='hhno',how='left')
bike_walk_trips = trip_person[trip_person['mode'].isin(['Bike','Walk'])]

df = bike_walk_trips.groupby(['hhno','pno']).count()
df = df.reset_index()
df = df[['hhno','pno']]
df['bike_walk'] = True

df = pd.merge(person_hh,df,on=['hhno','pno'], how='left')
df['bike_walk'] = df['bike_walk'].fillna(False)

In [117]:
pd.options.display.float_format = '{:,.1%}'.format
reg_active = pd.DataFrame(df.groupby('bike_walk').sum()['psexpfac']/df['psexpfac'].sum()).loc[True]['psexpfac']

In [118]:
df = pd.merge(df, special_taz, left_on='hhtaz', right_on='TAZ', how='left')
df = pd.merge(df, rgc_taz, left_on='hhtaz', right_on='taz', how='left')

In [119]:
_df = (df.groupby(['Low Income','bike_walk']).sum()['psexpfac']/df['psexpfac'].sum()).reset_index()
low_inc_active = _df.ix[(df['Low Income'] == 1) & (df['bike_walk']==True),'psexpfac'].tolist()[0]

In [127]:
_df = (df.groupby(['Minority','bike_walk']).sum()['psexpfac']/df['psexpfac'].sum()).reset_index()
minority_active = _df.ix[(df['Minority'] == 1) & (df['bike_walk']==True),'psexpfac'].tolist()[0]

In [128]:
_df = pd.DataFrame([reg_active,minority_active,low_inc_active])
_df.index = ['Region','People of Color','People of Low Income']
_df.columns = ['% Walks or Bikes for Transportation']
_df

,% Walks or Bikes for Transportation
Region,28.1%
People of Color,31.5%
People of Low Income,28.9%


### Average Time Spent Walking

In [ ]:
# pd.options.display.float_format = "{0:.2f}".format
# walk_trips = trip_person[trip_person['mode'] == 'Walk']
# daily_walk_times = walk_trips.groupby(['hhno','pno']).sum()['travtime']

In [7]:
daily_tot_time = pd.DataFrame(trip_person.groupby(['pno','hhno','mode']).sum()['travtime'])
daily_tot_time = daily_tot_time.reset_index()
daily_walk_time = daily_tot_time[daily_tot_time['mode'] == 'Walk']

In [21]:
# Total walk time across region divided by total number of people
reg_walk_time = daily_walk_time['travtime'].sum()/person['psexpfac'].sum()

In [26]:
# Total for low inc and for rgc
df = pd.merge(daily_tot_time,hh[['hhno','hhtaz']],on='hhno', how='left')
df = pd.merge(df,rgc_taz,left_on='hhtaz',right_on='taz',how='left')
df = pd.merge(df,special_taz,left_on='hhtaz',right_on='TAZ',how='left')

In [31]:
low_inc_walk_time = df[df['Low Income'] == 1]['travtime'].sum()/person['psexpfac'].sum()
minority_walk_time = df[df['Minority'] == 1]['travtime'].sum()/person['psexpfac'].sum()

In [35]:
df = pd.DataFrame([reg_walk_time,minority_walk_time,low_inc_walk_time])
df.columns = ['Average Minutes Walking per Day']
df.index = ['Region','People of Color','People of Low Income']
df

,Average Minutes Walking per Day
Region,23.655430
People of Color,34.847464
People of Low Income,32.405263


# Annual Out-of-Pocket Costs

In [46]:
annual_factor = 300
reg_costs = trip_hh[['hhno','travcost']].groupby('hhno').sum()['travcost'].mean()*annual_factor
low_inc_costs = trip_hh[trip_hh['Low Income'] == 1][['hhno','travcost']].groupby('hhno').sum()['travcost'].mean()*annual_factor
minority_costs = trip_hh[trip_hh['Minority'] == 1][['hhno','travcost']].groupby('hhno').sum()['travcost'].mean()*annual_factor

In [47]:
df = pd.DataFrame([reg_costs,minority_costs,low_inc_costs])
df.columns = ['Annual Out-of-Pocket Costs']
df.index = ['Region','People of Color','People of Low Income']
df

,Annual Out-of-Pocket Costs
Region,3483.524419
People of Color,3166.136363
People of Low Income,3055.232838


### Daily VMT

### Daily Total Trips

### Annual Delay per Capita